In [1]:
##Installing requirements
!pip install tensorflow==2.0.0beta numpy pandas scikit-learn

^C


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

In [3]:
#Implement GAN and VAE for generator class

class GAN(tf.keras.Model):
    def __init__(self, batch_size):
        super(GAN, self).__init__()
        self.batch_size = batch_size
        self.cross_entropy = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
        
        self.generator = self.make_generator_GAN()
        self.discriminator = self.make_discriminator_GAN()
        
    def make_generator_GAN(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(500, use_bias=False, input_shape=(100,)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())
          
        model.add(tf.keras.layers.Dense(250, use_bias=False))
        model.add(tf.keras.layers.Dense(200, use_bias=False))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())
    
        model.add(tf.keras.layers.Dense(50, use_bias=False))    
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())
    
        model.add(tf.keras.layers.Dense(131, activation="tanh", use_bias=False))      
        return model
    
    def make_discriminator_GAN(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(100, input_shape=(131,)))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(0.3))
          
        model.add(tf.keras.layers.Dense(50))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(0.3))
           
        model.add(tf.keras.layers.Dense(1))
         
        return model
    
    def generator_loss(self, fake_output):
        return self.cross_entropy(tf.ones_like(fake_output), fake_output)
    
    def discriminator_loss(self, real_output, fake_output):
        real_loss = self.cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = self.cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss
    
    @tf.function
    def train_step(self, dataset):
        noise = tf.random.normal([self.batch_size, 100])
    
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
          generated_data = self.generator(noise, training=True)
    
          real_output = self.discriminator(dataset, training=True)
          fake_output = self.discriminator(generated_data, training=True)
    
          gen_loss = self.generator_loss(fake_output)
          disc_loss = self.discriminator_loss(real_output, fake_output)
    
        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
    
        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
    
    def train(self, dataset, epochs):
        for epoch in range(epochs):
            for data_batch in dataset:
                self.train_step(data_batch)
                
    def generate(self, test_data, epochs):
        while epochs > 0:
            yield self.generator(test_data, training=False)
            epochs -= 1
        

In [4]:
class VAE(tf.keras.Model):
    def __init__(self, latent_dim, batch_size):
        super(VAE, self).__init__()
        self.batch_size = batch_size
        self.latent_dim = latent_dim
        self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.inference_optimizer = tf.keras.optimizers.Adam(1e-4)
        
        self.inference_net = self.inference_ae()
        self.generative_net = self.generative_ae()
    
    def inference_ae(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(32, use_bias=False, input_shape=(131,)))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dense(48, use_bias=False))
        model.add(tf.keras.layers.LeakyReLU())
              #tf.keras.layers.BatchNormalization(),
        model.add(tf.keras.layers.Dense(64, use_bias=False))
        model.add(tf.keras.layers.LeakyReLU())
              #tf.keras.layers.BatchNormalization(),
              # No activation
        model.add(tf.keras.layers.Dense(self.latent_dim + self.latent_dim, use_bias=False))
        return model
    
    def generative_ae(self):
        model = tf.keras.Sequential()

        model.add(tf.keras.layers.Dense(64, use_bias=False, input_shape=(None, self.latent_dim)))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dense(48, use_bias=False))
              #tf.keras.layers.BatchNormalization(),
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dense(32, use_bias=False))
        model.add(tf.keras.layers.LeakyReLU())
              #tf.keras.layers.BatchNormalization(),
              # No activation
        model.add(tf.keras.layers.Dense(131, use_bias=False))
        return model
        
    def sample(self, eps=None):
        if eps is None:
          eps = tf.random.normal(shape=(100, self.latent_dim))
        return self.decode(eps, train=False, apply_sigmoid=True)
    
    def encode(self, x, train=True):
        mean, logvar = tf.split(self.inference_net(x, training=train), num_or_size_splits=2, axis=1)
        return mean, logvar
    
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean
    
    def decode(self, z, train=True, apply_sigmoid=False):
        logits = self.generative_net(z, training=train)
        if apply_sigmoid:
          probs = tf.sigmoid(logits)
          return probs
    
        return logits
    
    def log_normal_pdf(self, sample, mean, logvar, raxis=1):
          log2pi = tf.math.log(2. * np.pi)
          return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis=raxis)

    
    def train_step(self, dataset):
    
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            dataset = tf.cast(dataset, tf.float32)
            mean, logvar = self.encode(dataset, True)
            z = self.reparameterize(mean, logvar)
            x_logit = self.decode(z, True)
        
            cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=dataset)
            logpx_z = -tf.reduce_sum(cross_ent, axis=1)
            logpz = self.log_normal_pdf(z, 0., 0.)
            logqz_x = self.log_normal_pdf(z, mean, logvar)
            loss = -tf.reduce_mean(logpx_z + logpz - logqz_x)
    
        gradients_of_generative = gen_tape.gradient(loss, self.generative_net.trainable_variables)
        gradients_of_inference = disc_tape.gradient(loss, self.inference_net.trainable_variables)
    
        self.generator_optimizer.apply_gradients(zip(gradients_of_generative, self.generative_net.trainable_variables))
        self.inference_optimizer.apply_gradients(zip(gradients_of_inference, self.inference_net.trainable_variables))
    
    def train(self, dataset, epochs):
        #self.model = VAE(self.latent_dim, self.batch_size)
        
        
        for epoch in range(epochs):

            for data_batch in dataset:
                self.train_step(data_batch)
                
    def generate(self, test_data, epochs):
        while epochs > 0:
            yield self.sample(test_data)
            epochs -= 1


In [5]:
#Generator:
class Generator(object):
    def __init__(self, data, p, batch_size):
        self.data = data
        self.p = p
        self.batch_size = batch_size
        self.gan = GAN(self.batch_size)
        self.vae = VAE(100, self.batch_size)
    
    def train(self, latent_dim, epochs):
        
        self.gan.train(self.data, epochs)
        
        self.vae.train(self.data, epochs)
        
    def generate(self, test_data, epochs):
        while epochs > 0:
            samp = np.random.random_sample()
            if self.p >= samp:
                yield next(self.gan.generate(test_data, epochs))
            else:
                yield next(self.vae.generate(test_data, epochs))
            epochs -= 1
        



In [6]:
#loading and preprocessing data
data = pd.read_csv("../data/ds_test.csv")
#MinMax Scaling for avoiding problems with VAE
scaler = MinMaxScaler()
data.loc[:,"amount"] = scaler.fit_transform(np.array(data.amount).reshape(-1,1))
#storing date and description for posterior mappings
date = data.date

desc = np.unique(data.description)

#store category as label and one-hot encode it

category = pd.get_dummies(data.category, prefix="category")
date = pd.get_dummies(data.date, prefix="date")
description = pd.get_dummies(data.description, prefix="description")

#remove currency and ID and old not onehot encoded columns
data = data.drop(["id", "currency", "category","description","date"], axis=1)

#add new columns
data = pd.concat([data, date, description, category], axis=1)

batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices(data.values).shuffle(36).batch(batch_size)

In [7]:
##The idea was to map back the one hot encode in order to structure the data generate (2 hours limit already)
generator = Generator(train_dataset, 0.6, 16)
generator.train(100, 100)


W0618 02:32:03.599898 14912 deprecation.py:323] From D:\DataScience\anaconda3\envs\AI\lib\site-packages\tensorflow\python\ops\nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
for i in range(70):
    next(generator.generate(tf.random.normal([32, 100]), 50))